In [1]:
from numpy import random
from copy import deepcopy
from bisect import insort

rng = random.default_rng(123)

In [2]:
def choose_random_questions(
        options: dict[str, list[int]], 
        *, 
        num_questions: int = 4, 
        modify_options: bool = True
    ) -> dict[str, list[int]]:
    '''
    Sample `num_questions` from a dictionary `options`. Options is expected to be structued as `'Chapter': [Questions]`.
    By default, the passed dictionary will be modified. Set `modify_options` if you would prefer this script to work on
    a (deep)copy of the dictionary.

    This works by sampling a chapter (without replacement) and then sampling a question (without replacement).

    Noteworthy behavior:
        - If `num_questions > # of chapters`, then the chapters are refreshed.
        - If all the questions from a chapter is sampled, then that chapter is removed entirely.
        - The same question can never be chosen twice (i.e. questions are always sampled without replacement).
    '''
    options = options if modify_options else deepcopy(options)
    chosen_questions = {
        key: []
        for key in options
    } 

    # strategy: run through the algorithm and perform as many full samples as needed
    # then perform a final run where the remainder is sampled
    for i in range(num_questions // len(options)):
        for chapter in options:
            index = rng.integers(len(options[chapter])) # choose a random index
            insort(
                chosen_questions[chapter], 
                options[chapter].pop(index)
            ) # remove question from chapter and insert it (preserve order)
            if not options[chapter]:
                del options[chapter]
    
    # perform final run
    remaining_chapters = rng.choice(
        list(options.keys()), 
        size=num_questions % len(options), 
        replace=False
    )
    for chapter in remaining_chapters:
        insort(
            chosen_questions[chapter],
            int(rng.choice(options[chapter]))
        )

    return chosen_questions

In [3]:
repr(choose_random_questions({
    'Ch 3': [1,4,14], 
    'Ch 20': [5,6,18,21],
    'Ch 21': [9,11,17,23]
}))

"{'Ch 3': [1, 14], 'Ch 20': [18], 'Ch 21': [17]}"